# 🧠 05 – LLM-Inspired Text Reasoning Features (Chain-of-Thought)

โปรเจกต์นี้สร้าง “ระบบให้เหตุผลแบบ LLM” สำหรับวิเคราะห์ข้อความร้องเรียนของ Traffy จาก Comment  

---

## 🎯 เป้าหมาย
สร้าง feature ใหม่จาก `comment` เพื่อใช้ทำนาย breach โดยใช้แนวคิดของ LLM เช่น

- สร้าง reasoning chain (ลำดับความคิด)
- ให้เหตุผลเป็นภาษาคน
- สร้าง risk score แบบมีหลักฐาน
- แปลง reasoning เป็น feature ที่โมเดล ML ใช้ได้

---

## 🚀 ภาพรวมวิธีที่เราใช้ (LLM-inspired pipeline)

### 1) TF-IDF + Logistic Regression → หา “risk char-grams”
เหมือนให้ LLM วิเคราะห์ว่า “คำไหนสื่อความเสี่ยง”

- สร้าง TF-IDF (ngram 3–5)
- เทรน Logistic Regression บน label breach
- ดึงคำ/สำนวนที่มีน้ำหนักสูงสุดต่อการทำให้ breach เพิ่ม/ลด

ผลลัพธ์:
- `risk_words_global`
- ใช้เป็น signal ใน reasoning chain

---

### 2) เขียน “reasoning chain” เอง 
สร้างฟังก์ชัน `reason_about_risk(row)` ที่ประมวลผลหลายปัจจัย เช่น:

- urgency  
- std_type  
- เวลาที่แจ้ง (กลางคืน/เช้าเร่งด่วน)  
- risk char-grams  
- ความยาว comment  
- ความชื้นจากสภาพอากาศ  

ในแต่ละปัจจัย:
- เพิ่ม **คะแนนเสี่ยง** (rule_risk_score_raw)
- เก็บ **เหตุผลเป็นข้อความ** (rule_risk_reason)

เหมือน LLM ที่สรุปเป็น step-by-step reasoning

---

### 3) Normalize + Quantile Calibration → ตัดระดับความเสี่ยง
แทนที่จะกำหนด threshold แบบสุ่ม → ใช้ **quantile ของ distribution จริง**

- `q_med` = 0.50 (median)  
- `q_high` = 0.75 (top 25%)  

แปลงเป็นระดับ:
- **low**
- **medium**
- **high**

และสร้างคอลัมน์:
- `rule_risk_score_raw`
- `rule_risk_level`
- `rule_risk_reason`

---

## 📊 คุณสมบัติที่ได้ (LLM-like Features)

| Feature | ชนิด | ใช้ทำอะไร |
|--------|------|-----------|
| `rule_risk_score_raw` | numeric | คะแนนเสี่ยงแบบ reasoning chain |
| `rule_risk_level` | categorical | low / medium / high (จาก quantile) |
| `rule_risk_reason` | text | อธิบายเหตุผลของระบบ (human-readable) |



In [18]:
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


In [19]:
completed_path = "../data/traffy_completed_for_model_with_weather.csv"
pending_path   = "../data/traffy_pending_for_predict_with_weather.csv"

df_completed = pd.read_csv(completed_path)
df_pending   = pd.read_csv(pending_path)


In [20]:
df_completed.head(3)

,ticket_id,std_type,district,subdistrict,province_clean,urgency,comment,comment_length,day_of_week,hour_of_day,...,lng,lat,SLA_days,breach,temp_high,temp_low,humidity_high,humidity_low,pressure_high,pressure_low
0,2024-CVGNCL,road,ราชเทวี,ถนนพญาไท,กรุงเทพมหานคร,high,ปัญหา: บริเวณหน้าบ้านเลขที่ 425/1-2 พบคนเรร่อน...,351,2,2,...,100.54204,13.75420,12.461950,1,36.0,24.0,100.0,47.0,1012.0,1001.0
1,2024-ATFNH8,cleaning,ลาดกระบัง,คลองสองต้นนุ่น,กรุงเทพมหานคร,mid,ขยะไม่เก็บ,10,3,8,...,100.73608,13.73114,11.866649,0,35.0,21.0,100.0,41.0,1016.0,1006.0
2,2024-ET2E9B,road,คลองสามวา,บางชัน,กรุงเทพมหานคร,high,ไฟส่องสว่าง บริเวณเกาะกลางดับ ถนนสามวา (สามวา ...,54,6,7,...,100.72685,13.84308,12.461950,1,40.0,24.0,100.0,29.0,1014.0,1001.0


In [21]:
df_pending.head(3)

,ticket_id,std_type,district,subdistrict,province_clean,urgency,comment,comment_length,day_of_week,hour_of_day,...,year,lng,lat,SLA_days,temp_high,temp_low,humidity_high,humidity_low,pressure_high,pressure_low
0,ZN3ZZV,road,ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,high,ปัญหา: ริมถนนดังกล่าว บริเวณหน้าเซ่เวน อีเลฟเว...,424,5,18,...,2024,100.59014,13.80482,12.46195,34.0,19.0,94.0,35.0,1018.0,1006.0
1,2024-N9H4PR,road,พญาไท,สามเสนใน,กรุงเทพมหานคร,high,โคมไฟที่ติดตั้งใต้ท้องสะพาน ของทางพิเศษเฉลิมมห...,723,5,11,...,2024,100.54818,13.76384,12.46195,39.0,25.0,94.0,20.0,1016.0,1005.0
2,2024-GDN42R,road,หนองจอก,คลองสิบสอง,กรุงเทพมหานคร,high,ไฟดับ ถนนประชาสำราญ บริเวณตรงข้ามร้าน น้องหมิว...,98,5,3,...,2024,100.86285,13.89961,12.46195,34.0,19.0,94.0,35.0,1018.0,1006.0


In [22]:
def clean_comment(text: str) -> str:
    """ทำความสะอาดข้อความ comment แบบเบา ๆ (ไม่ทำ stemming/stopword เพราะเป็นไทยผสมอังกฤษ)"""
    if pd.isna(text):
        return ""
    text = str(text)
    # ลบ \n, \r, แทนด้วย space
    text = re.sub(r"[\r\n]+", " ", text)
    # ลบ space ซ้ำ
    text = re.sub(r"\s+", " ", text)
    return text.strip()

df_completed["comment_clean"] = df_completed["comment"].apply(clean_comment)
df_pending["comment_clean"]   = df_pending["comment"].apply(clean_comment)


In [23]:
df_text = df_completed[["comment_clean", "breach"]].dropna(subset=["comment_clean", "breach"]).copy()
df_text["breach"] = df_text["breach"].astype(int)

df_text.head(3)

,comment_clean,breach
0,ปัญหา: บริเวณหน้าบ้านเลขที่ 425/1-2 พบคนเรร่อน...,1
1,ขยะไม่เก็บ,0
2,ไฟส่องสว่าง บริเวณเกาะกลางดับ ถนนสามวา (สามวา ...,1


In [24]:
tfidf = TfidfVectorizer(
    analyzer="char_wb",   
    ngram_range=(3, 5),   # 3–5 ตัวอักษร
    min_df=20,            # อย่างน้อย 20 เคส
    max_features=5000
)

X = tfidf.fit_transform(df_text["comment_clean"])
y = df_text["breach"].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

clf = LogisticRegression(
    max_iter=1000,
    n_jobs=-1,
)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, digits=4))


/opt/anaconda3/envs/dede-cedt/lib/python3.11/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)


              precision    recall  f1-score   support

           0     0.7636    0.9619    0.8514     16482
           1     0.5760    0.1481    0.2356      5760

    accuracy                         0.7511     22242
   macro avg     0.6698    0.5550    0.5435     22242
weighted avg     0.7150    0.7511    0.6919     22242



In [25]:
feature_names = np.array(tfidf.get_feature_names_out())
coefs = clf.coef_[0]

# คำที่ดัน breach ให้สูงขึ้น (coef > 0)
top_n = 50

top_risk_idx = np.argsort(coefs)[-top_n:]
top_safe_idx = np.argsort(coefs)[:top_n]

top_risk_words = feature_names[top_risk_idx]
top_safe_words = feature_names[top_safe_idx]

print("🔥 Top risk char-grams (ดันให้ breach สูงขึ้น):")
print(top_risk_words[::-1])  # reverse ให้ตัวใหญ่สุดอยู่บน

print("\n🧊 Top safe char-grams (ดันให้ breach ต่ำลง):")
print(top_safe_words)


🔥 Top risk char-grams (ดันให้ breach สูงขึ้น):
['ถนน' 'รัก' 'งรั' 'างร' 'ัก ' 'ีเส' 'งค์' 'ดับ' 'ือง ' 'ระย' 'มมา' 'หมด'
 'ดวง' 'าลา' 'ยงเ' 'บาท' 'หาย' 'กร ' 'ายจ' 'แยก' 'ยเห' 'ุมว' 'ปูน' 'นทร'
 'นสะ' 'พัง' ' เสา' ' ตั' 'นาด' 'สาม' 'อบก' ' ทา' 'ชื่' 'ขัง' 'มีแ' 'กระ'
 'วัน ' 'หยุ' ' 4 ' 'มถึง' 'เชื' ' จา' 'พชร' 'มถึ' ' รา' 'เพช' 'ามว' 'ใต้'
 'เพชร' 'งท่']

🧊 Top safe char-grams (ดันให้ breach ต่ำลง):
['เผา' 'ขี่' 'สาย' 'ถัง' 'แมว' 'ล้อ' 'ายก' 'เร่' 'บางแ' 'วัด' 'ใคร' 'พบรถ'
 'ิดป' 'านใ' 'าสา' '้าย ' 'ี่บ' 'ายต' 'ราบ' ' ตล' 'นเส' 'รถบ' 'งแค' 'มเส'
 'อีก' 'าย ' 'มิน' 'เท้า ' 'ท้า ' 'ควั' 'ุ่ง' 'ทอง' 'กาศ' 'าท่อร' 'ควัน'
 '่ที' 'าวเ' 'แก่' '่บน' 'จัด' ' จอดร' 'ยาว' 'กอง' 'วาง' ' สา' 'ดให' ' พบ'
 'เสียง' 'วจร' '30 ']


In [26]:
risk_words_global = set(top_risk_words.tolist())
safe_words_global = set(top_safe_words.tolist())
len(risk_words_global), len(safe_words_global)

(50, 50)

In [27]:
HIGH_RISK_TYPES = {"road", "flood", "lighting", "safety"}
MID_RISK_TYPES  = {"noise", "canal", "cleaning"}
LOW_RISK_TYPES  = {"animal", "other", "unknown"}

def reason_about_risk(row,
                      risk_grams=risk_words_global,
                      high_types=HIGH_RISK_TYPES,
                      mid_types=MID_RISK_TYPES):
    """
    สร้าง reasoning chain เป็น:
        - คะแนนเชิงตัวเลข (rule_risk_score_raw)
        - เหตุผลแบบภาษาคน (rule_risk_reason)
    ***ยังไม่ตัดสินใจ level ที่นี่*** เพื่อให้ไป calibrate ด้วย quantile ทีหลัง
    """
    score = 0.0
    reasons = []

    std_type = str(row.get("std_type", "unknown"))
    urgency = str(row.get("urgency", "low"))
    hour    = int(row.get("hour_of_day", -1))
    comment_clean = str(row.get("comment_clean", ""))
    clen   = int(row.get("comment_length", 0))
    hum_h  = row.get("humidity_high", np.nan)

    # 1) urgency
    if urgency == "high":
        score += 1.0
        reasons.append("ระดับความเร่งด่วน = HIGH (ระบบกำหนด) → เสี่ยงต่อการ breach")
    elif urgency == "mid":
        score += 0.5
        reasons.append("ระดับความเร่งด่วน = MID → มีโอกาส breach ปานกลาง")
    else:
        reasons.append("ระดับความเร่งด่วน = LOW → ความเสี่ยงโดยพื้นฐานไม่สูงมาก")

    # 2) std_type
    if std_type in high_types:
        score += 0.8
        reasons.append(f"หมวดปัญหา {std_type} เป็นหมวดเสี่ยงสูง (เช่น น้ำท่วม/ความปลอดภัย)")
    elif std_type in mid_types:
        score += 0.4
        reasons.append(f"หมวดปัญหา {std_type} เป็นหมวดเสี่ยงกลางๆ → เสี่ยงปานกลาง")
    else:
        reasons.append(f"หมวดปัญหา {std_type} อยู่ในกลุ่มความเสี่ยงต่ำกว่า")

    # 3) เวลาแจ้ง 
    if 0 <= hour <= 5 or hour >= 22:
        if std_type in {"lighting", "safety", "road"}:
            score += 0.7
            reasons.append(
                f"แจ้งในช่วงกลางคืน/เช้ามืด (hour={hour}) + หมวด {std_type} → อาจกระทบความปลอดภัยมาก"
            )
        else:
            score += 0.4
            reasons.append(
                f"แจ้งในช่วงกลางคืน/เช้ามืด (hour={hour}) → มีโอกาสจัดการได้ช้ากว่าช่วงกลางวัน"
            )
    elif 6 <= hour <= 9:
        score += 0.2
        reasons.append(
            f"แจ้งช่วงเช้า (hour={hour}) ซึ่งเป็นช่วงงานล้นของหน่วยงาน → มีโอกาส delay เล็กน้อย"
        )

    # 4) คำ/character-grams เสี่ยงจาก text model
    hit_grams = []
    for g in risk_grams:
        if g in comment_clean:
            hit_grams.append(g)
            # พอเจอสัก 1–2 ตัวก็พอแล้ว ไม่ต้องบวกทีละ 1 ให้ล้น
            if len(hit_grams) >= 2:
                break

    if len(hit_grams) > 0:
        score += 0.7
        reasons.append(
            f"พบ pattern ตัวอักษรที่ model เรียนรู้ว่าเกี่ยวข้องกับ breach (เช่น {', '.join(hit_grams)})"
        )

    # 5) ความยาว comment
    if clen > 300:
        score += 0.5
        reasons.append(
            f"คำอธิบายยาวมาก (length={clen}) → มักเป็นเคสซับซ้อน/ร้องเรียนหลายประเด็น → เสี่ยง breach สูง"
        )
    elif clen > 150:
        score += 0.2
        reasons.append(
            f"คำอธิบายยาว (length={clen}) → มีโอกาสเป็นเคสซับซ้อน"
        )

    # 6) Weather
    if not pd.isna(hum_h) and hum_h >= 95:
        score += 0.3
        reasons.append(
            f"ความชื้นสูงมาก (humidity_high={hum_h}) → สภาพอากาศอาจทำให้การปฏิบัติงานล่าช้า"
        )

    reason_text = " | ".join(reasons)
    return score, reason_text

# -----------------------
# Apply กับ df_completed
# -----------------------
scores_c, reasons_c = [], []

for _, row in df_completed.iterrows():
    s, rs = reason_about_risk(row)
    scores_c.append(s)
    reasons_c.append(rs)

df_completed["rule_risk_score_raw"] = scores_c
df_completed["rule_risk_reason"]    = reasons_c

df_completed["rule_risk_score_raw"].describe()



count    111210.000000
mean          2.479369
std           0.905999
min           0.000000
25%           2.000000
50%           2.700000
75%           3.200000
max           4.000000
Name: rule_risk_score_raw, dtype: float64

In [28]:
# เลือก quantile สำหรับแบ่งระดับ
q_high = df_completed["rule_risk_score_raw"].quantile(0.75)   # top 25% เป็น high
q_med  = df_completed["rule_risk_score_raw"].quantile(0.50)   # 50–75% เป็น medium

print("q_med =", q_med, "q_high =", q_high)

def score_to_level(s, q_med=q_med, q_high=q_high):
    if s >= q_high:
        return "high"
    elif s >= q_med:
        return "medium"
    else:
        return "low"

df_completed["rule_risk_level"] = df_completed["rule_risk_score_raw"].apply(score_to_level)


q_med = 2.7 q_high = 3.2


In [29]:
print(df_completed["rule_risk_level"].value_counts(normalize=True))
print( )
print(
    df_completed.groupby("rule_risk_level")["breach"]
    .mean()
    .sort_index()
)


rule_risk_level
low       0.473752
high      0.264284
medium    0.261964
Name: proportion, dtype: float64

rule_risk_level
high      0.316457
low       0.201970
medium    0.304122
Name: breach, dtype: float64


In [30]:
# -----------------------
# Apply กับ df_pending
# -----------------------
scores_p, reasons_p = [], []

for _, row in df_pending.iterrows():
    s, rs = reason_about_risk(row)
    scores_p.append(s)
    reasons_p.append(rs)

df_pending["rule_risk_score_raw"] = scores_p
df_pending["rule_risk_reason"]    = reasons_p

df_pending["rule_risk_level"] = df_pending["rule_risk_score_raw"].apply(score_to_level)

df_pending[["ticket_id", "std_type", "urgency",
            "rule_risk_score_raw", "rule_risk_level"]].head()


,ticket_id,std_type,urgency,rule_risk_score_raw,rule_risk_level
0,ZN3ZZV,road,high,3.0,medium
1,2024-N9H4PR,road,high,3.0,medium
2,2024-GDN42R,road,high,3.2,high
3,2024-LNHRXD,noise,mid,1.6,low
4,2024-LMCD6M,unknown,low,0.2,low


In [31]:
df_completed.to_csv("../data/traffy_completed_for_model_with_weather_and_llm.csv", index=False)
df_pending.to_csv("../data/traffy_pending_for_predict_with_weather_and_llm.csv", index=False)
df_completed.sample(100,random_state=42).to_csv("../data_samples/04-traffy_completed_for_model_with_weather_and_llm.csv", index=False)
df_pending.sample(100,random_state=42).to_csv("../data_samples/04-traffy_pending_for_predict_with_weather_and_llm.csv", index=False)